In [0]:
# =============================================================================
# CAMADA SILVER - SETS - MAGIC: THE GATHERING
# =============================================================================
"""
Script Python para processamento da tabela TB_REF_SILVER_SETS
Transformação e limpeza de dados da Bronze para Silver

USO DE SILVER_UTILS.PY:
- Centralização de funções comuns
- Padronização de processamento
- Redução de código duplicado
"""

# =============================================================================
# BIBLIOTECAS UTILIZADAS
# =============================================================================
import logging
from datetime import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import *

# =============================================================================
# CARREGAMENTO DO MÓDULO UTILITÁRIO
# =============================================================================
# Importar funções do silver_utils usando %run (Databricks)
%run ./silver_utils

# =============================================================================
# CONFIGURAÇÃO INICIAL
# =============================================================================
def get_secret(secret_name, default_value=None):
    try:
        return dbutils.secrets.get(scope="mtg-pipeline", key=secret_name)
    except:
        if default_value is not None:
            print(f"Segredo '{secret_name}' não encontrado, usando valor padrão")
            return default_value
        else:
            print(f"Segredo obrigatório '{secret_name}' não encontrado")
            raise Exception(f"Segredo '{secret_name}' não configurado")

def setup_logging():
    """Configura logging para o script"""
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s'
    )
    return logging.getLogger(__name__)

def transform_sets_silver(df):
    """
    Transformação específica para tabela Sets
    Inclui lógica específica para Sets além das transformações padrão
    """
    if not df:
        return None
    
    logger = logging.getLogger(__name__)
    logger.info("Iniciando transformações específicas para Sets...")
    
    # Aplicar limpeza padrão
    df = apply_standard_cleaning(
        df,
        name_columns=["NME_SET", "NME_TYPE", "NME_SOURCE"],
        desc_columns=["NME_SOURCE"]
    )
    
    # Conversão de datas
    df = df.withColumn("DT_RELEASE", to_date(col("DT_RELEASE")))
    
    # Adicionar colunas de particionamento
    df = add_partition_columns(df, "RELEASE_YEAR", "RELEASE_MONTH")
    
    # Seleção final de colunas
    colunas_finais = [
        "COD_SET", "NME_SET", "NME_TYPE", "DT_RELEASE", 
        "FLG_ONLINE_ONLY", "NME_SOURCE", "RELEASE_YEAR", "RELEASE_MONTH"
    ]
    
    # Filtrar colunas que existem no DataFrame
    colunas_disponiveis = [c for c in colunas_finais if c in df.columns]
    df_final = df.select(*colunas_disponiveis)
    
    logger.info(f"Transformação Sets concluída: {df_final.count()} registros")
    return df_final

# =============================================================================
# CONFIGURAÇÃO
# =============================================================================

# Configuração manual 
config = create_manual_config("magic_the_gathering", get_secret("s3_bucket"))

# Setup Unity Catalog
setup_unity_catalog(config['catalog_name'], config['schema_silver'])



In [0]:
# =============================================================================
# PROCESSAMENTO USANDO SILVER_UTILS
# =============================================================================
# Criar processor
processor = SilverTableProcessor("TB_REF_SILVER_SETS", config)

# Extração da Bronze
df_bronze = processor.extract_from_bronze("TB_BRONZE_SETS")

# Aplicar transformação específica
df_silver = processor.transform_data(df_bronze, transform_sets_silver)

# Salvar na Silver com merge incremental
processor.save_silver_table(
    df_silver, 
    partition_cols=["RELEASE_YEAR", "RELEASE_MONTH"],
    key_column="COD_SET"
)

# =============================================================================
# VALIDAÇÃO E LOGS
# =============================================================================
if df_silver:
    print(f"Processamento concluído com sucesso!")
    print(f"Registros processados: {df_silver.count()}")
    print(f"Colunas finais: {df_silver.columns}")
else:
    print("Falha no processamento - DataFrame vazio") 